In [1]:
!pip install chromadb

In [2]:
import pandas as pd

In [5]:
url = "https://raw.githubusercontent.com/jogfx/MLops-exam/main/job_listings_cleaned.csv"
df = pd.read_csv(url)

In [7]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1459644,SROspecialist,Vil du vre med til at udvikle og optimere aflb...,2024-04-11,København S,https://www.jobindex.dk/vis-job/h1459644
1,h1459642,Regnskabsfrerbogholder til Det Flles Regnskabs...,Det Flles Regnskabskontor i Silkeborg og Skand...,2024-04-11,Silkeborg,https://www.jobindex.dk/vis-job/h1459642
2,h1459641,Accountant med speciale i administration af le...,I General Accounting fr vi snarest muligt brug...,2024-04-11,Allerød,https://www.jobindex.dk/vis-job/h1459641
3,h1459638,Socialrdgiver til den sikrede institution Bakk...,Er du interesseret i at indg i et samarbejde o...,2024-04-11,Nykøbing Sj,https://www.jobindex.dk/vis-job/h1459638
4,h1459635,Area Sales Manager,The role of the Area Sales Manager Orkla Healt...,2024-04-11,unknown,https://www.jobindex.dk/vis-job/h1459635


In [8]:
# Importing necessary libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
import nltk

In [9]:
# Downloading necessary files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\olive\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
# Function to preprocess text
def tokenize_and_lowercase(text):
    return [token.lower() for token in word_tokenize(text)]

def remove_stop_words(tokens):
    return [word for word in tokens if word not in stop_words]

def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def preprocess_text(text):
    tokens = tokenize_and_lowercase(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_words(tokens)
    return ' '.join(tokens)

In [11]:
# Preprocessing text columns
text_columns = ['Title', 'Description']

for column in text_columns:
    df[column] = df[column].astype(str).apply(preprocess_text)

In [12]:
df.head()

,Entry_id,Title,Description,Published Date,Location,URL
0,h1459644,srospecialist,vil du vre med til udvikle og optimere aflbssy...,2024-04-11,København S,https://www.jobindex.dk/vis-job/h1459644
1,h1459642,regnskabsfrerbogholder til det flles regnskabs...,det flles regnskabskontor silkeborg og skander...,2024-04-11,Silkeborg,https://www.jobindex.dk/vis-job/h1459642
2,h1459641,accountant med speciale administration af lejeml,general accounting fr vi snarest muligt brug e...,2024-04-11,Allerød,https://www.jobindex.dk/vis-job/h1459641
3,h1459638,socialrdgiver til den sikrede institution bakk...,er du interesseret indg et samarbejde om vider...,2024-04-11,Nykøbing Sj,https://www.jobindex.dk/vis-job/h1459638
4,h1459635,area sale manager,role area sale manager orkla health internatio...,2024-04-11,unknown,https://www.jobindex.dk/vis-job/h1459635


In [13]:
import csv

In [23]:
# Writing the preprocessed data to a CSV file
with open('job_listings_cleaned.csv') as f:
    lines = csv.reader(f)

    documents = []
    metadatas = []
    ids = []
    id = 1

    for i, line in enumerate(lines):
        if i == 0:
            continue
    
        documents.append(line[1])
        metadatas.append({'Entry_id': line[0]})
        ids.append(str(id))
        id += 1

In [25]:
# Importing the Chroma SDK
import chromadb
chroma_client = chromadb.Client()

In [27]:
# Creating a collection
collection = chroma_client.create_collection('job_listings')

In [29]:
# Adding documents to the collection
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

C:\Users\olive\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:03<00:00, 21.1MiB/s]


In [46]:
results = collection.query(query_texts=['data scientist'], n_results=5)

In [47]:
results

{'ids': [['5510', '2448', '11739', '11405', '9194']],
 'distances': [[0.0,
   0.30933678150177,
   0.3827909529209137,
   0.3827909529209137,
   0.38855043053627014]],
 'metadatas': [[{'Entry_id': 'h1454261'},
   {'Entry_id': 'h1457323'},
   {'Entry_id': 'r12404603'},
   {'Entry_id': 'r12404492'},
   {'Entry_id': 'r12414784'}]],
 'embeddings': None,
 'documents': [['Data Scientist',
   'Data science student',
   'Senior Data Scientist',
   'Senior Data Scientist',
   'Data Analyst']],
 'uris': None,
 'data': None}